# Practica APA - predicció superfície cremada d'un incendi
Sergi Curto Panisello,
Joan Melchor Lladó

## Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from time import time
from datetime import timedelta

everything = False

## Obtenim les dades
Procedim a agafar les dades en format csv descarregades de https://datos.civio.es/dataset/todos-los-incendios-forestales/

In [2]:
df = pd.read_csv('fires-all.csv', index_col='id')
print(df.describe())

         superficie           lat           lng  latlng_explicit  \
count  82640.000000  82616.000000  82616.000000     82640.000000   
mean      19.888085     41.763721     -5.664360         0.741626   
std      223.787536      2.019672      4.394031         0.437743   
min        1.000000      0.490720  -1000.000000         0.000000   
25%        1.500000     40.876544     -7.257298         0.000000   
50%        3.000000     42.371123     -6.019478         1.000000   
75%        7.150000     43.143145     -4.303078         1.000000   
max    28879.100000     87.824157    242.755603         1.000000   

        idcomunidad   idprovincia   idmunicipio         causa  causa_supuesta  \
count  82640.000000  82640.000000  82640.000000  82640.000000         46465.0   
mean       7.850278     28.411375     77.795946      3.695051             1.0   
std        5.419922     11.727155     98.867769      1.032923             0.0   
min        1.000000      1.000000      1.000000      1.000000  

## Preprocessing

### Tractament de missing values

In [3]:
# Columnes amb missing values, també es pot veure al describe del
# dataset a les variables que no tinguin un count de 82640
print(df.columns[df.isnull().any()].tolist())

['lat', 'lng', 'causa_supuesta', 'muertos', 'heridos', 'gastos', 'perdidas']


#### Arreglar lat i long

Per arreglar lat i long ho fem en un altre document "corregirCoordenades.py", ja que és un procés més lent.
Es dedica a comprovar les coordenades de cada instància per veure si es corresponen a un diccionari generat
anteriorment amb el fitxer "obtenirCoordenades.py". Hi ha una explicació més extensa a la documentació.

In [4]:
df = pd.read_csv('coordsCorregides.csv', index_col='id')
# Comprovem que ja no queden nulls a coordenades
print(df.columns[df.isnull().any()].tolist())

['causa_supuesta', 'muertos', 'heridos', 'gastos', 'perdidas']


#### Arreglar muertos, heridos y causa_supuesta

In [5]:
# Comencem per emplenar els missing values de muertos i heridos ja que
# si no hi han dades suposarem que són 0.
df['muertos'].fillna(0, inplace=True)
df['heridos'].fillna(0, inplace=True)
# Sobre la "causa supuesta" és normal que hi hagin instàncies sense valor ja que és o 1 o nan per tant els posem a 0
df['causa_supuesta'].fillna(0, inplace=True)


#### Comprovar provincies i comunitats

In [6]:
import pandasql as ps

q1 = "SELECT municipio  FROM df GROUP BY municipio HAVING COUNT(DISTINCT idprovincia) > 1;"
q2 = "SELECT idprovincia FROM df GROUP BY idprovincia HAVING COUNT(DISTINCT idcomunidad) > 1;"

print("Municipio a la mateixa provincia", ps.sqldf(q1))
print("Provincia a la mateixa comunitat", ps.sqldf(q2))

Municipio a la mateixa provincia Empty DataFrame
Columns: [municipio]
Index: []
Provincia a la mateixa comunitat Empty DataFrame
Columns: [idprovincia]
Index: []


#### Arreglar gastos y perdidas

Per a fer aquesta part hem de netejar una mica més el dataset, la columna de "idmunicipio" és incorrecte, ja que
diferents municipis comparteixen el mateix id tot i està a cada punta del territori. També eliminem la columna municipio
ja que amb la latitud i longitud ja sabem on està localitzat l'incendi.

Primer separem target del dataset, volem predir la superfície cremada donat un incendi per
tant separem aquesta variable de la resta.

https://medium.com/@kyawsawhtoon/a-guide-to-knn-imputation-95e2dc496e

In [7]:
# Arreglar valors massa petits posant-lo a NaN
df.at[df.loc[((df["gastos"] < 25) & (df["gastos"] != 0))].index, 'gastos'] = np.NaN
df.at[df.loc[((df["perdidas"] < 25) & (df["perdidas"] != 0))].index, 'perdidas'] = np.NaN
# Valors massa extranys
df.at[df.loc[(df["perdidas"] == 999999)].index, 'perdidas'] = np.NaN
df.at[df.loc[(df["gastos"] == 999999)].index, 'gastos'] = np.NaN

# Convertir fecha en un numero en comptes de string
df["fecha"] = df["fecha"].apply(lambda f: f.replace('-', ''))
# Esborrem columnes innecessàries
df.drop(['idmunicipio', 'municipio', 'idcomunidad'], axis=1, inplace=True)

In [8]:
# Corregir valors de time ext i time control ja que no podem tenir que s'ha tardat més en controlar
# que en extendre. També treiem els que tenen valor de 0 ja que sino, no podrem aplicar la següent formula
df.at[df.loc[(df["time_ctrl"] > df["time_ext"])].index, ['time_ctrl', 'time_ext']] = np.NaN
df.at[df.loc[(df["time_ctrl"] == 0) | (df["time_ext"] == 0)].index, ['time_ctrl', 'time_ext']] = np.NaN
# Comprovem que el temps de control i d'extensió és adequat per les hectàrees de l'incendi
df.at[df.loc[(df["time_ctrl"] / df["superficie"] < 20)].index, ['time_ctrl', 'time_ext']] = np.NaN

df = pd.get_dummies(data=df, columns=["causa", "causa_desc", "idprovincia"])


In [9]:
start = time()
print(df.columns[df.isnull().any()].tolist())

from sklearn.preprocessing import MinMaxScaler
scalerMinMaxKNNImputer = MinMaxScaler()
df = pd.DataFrame(scalerMinMaxKNNImputer.fit_transform(df), columns=df.columns)

if everything:
    # # # Apliquem KNN per tal d'emplenar els missing values
    from sklearn.impute import KNNImputer
    imputer = KNNImputer(n_neighbors=1, copy=False)
    pd.DataFrame(imputer.fit_transform(df), columns=df.columns)

    print("--- %s seconds ---" % (time.time() - start))

    df.describe()
    print(df.columns[df.isnull().any()].tolist())

    df.to_csv("noNaNDataFrame.csv")
    print(df.head())
    print(df.describe())

df = pd.read_csv('noNaNDataFrame.csv', index_col=0)
df = pd.DataFrame(scalerMinMaxKNNImputer.inverse_transform(df), columns=df.columns)

['time_ctrl', 'time_ext', 'gastos', 'perdidas']


## Feature Extraction

In [10]:
df['PeoplePerHour'] = df.personal/(df.time_ctrl/60)
df['MediosPerHour'] = df.medios/(df.time_ctrl/60)
print(df['PeoplePerHour'])
print(df['MediosPerHour'])

0         4.000000
1         0.872727
2         1.777778
3         1.074627
4        14.666667
           ...    
82522    24.761905
82523     9.363057
82524    13.043478
82525    22.352941
82526    11.473684
Name: PeoplePerHour, Length: 82527, dtype: float64
0        0.800000
1        0.218182
2        0.444444
3        0.179104
4        2.666667
           ...   
82522    2.857143
82523    0.955414
82524    1.739130
82525    4.705882
82526    1.017544
Name: MediosPerHour, Length: 82527, dtype: float64


## Scaling and normalization of Dataset

També fem el train, test, validation split

In [11]:
from sklearn.model_selection import train_test_split, cross_val_score
X = df.drop("superficie", axis=1)
y = df["superficie"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.33, random_state=42)

# StandardScaler
from sklearn.preprocessing import StandardScaler
stdScaler = StandardScaler()
stdScaler.fit(X_train)
aux_train = stdScaler.transform(X_train.values)
X_train_std = pd.DataFrame(aux_train, index=X_train.index, columns=X_train.columns)
aux_test = stdScaler.transform(X_test.values)
X_test_std = pd.DataFrame(aux_test, index=X_test.index, columns=X_test.columns)
aux_val = stdScaler.transform(X_val.values)
X_val_std = pd.DataFrame(aux_val, index=X_val.index, columns=X_val.columns)

# MinMaxScaler
from sklearn.preprocessing import MinMaxScaler
scalerMinMaxKNNImputer = MinMaxScaler()
scalerMinMaxKNNImputer.fit(X_train)
aux_train = scalerMinMaxKNNImputer.transform(X_train.values)
X_train_minMax = pd.DataFrame(aux_train, index=X_train.index, columns=X_train.columns)
aux_test = scalerMinMaxKNNImputer.transform(X_test.values)
X_test_minMax = pd.DataFrame(aux_test, index=X_test.index, columns=X_test.columns)
aux_val = scalerMinMaxKNNImputer.transform(X_val.values)
X_val_minMax = pd.DataFrame(aux_val, index=X_val.index, columns=X_val.columns)

from sklearn.preprocessing import Normalizer
normalizer = Normalizer()
normalizer.fit(X_train)
aux_train = scalerMinMaxKNNImputer.transform(X_train.values)
X_train_norm = pd.DataFrame(aux_train, index=X_train.index, columns=X_train.columns)
aux_test = scalerMinMaxKNNImputer.transform(X_test.values)
X_test_norm = pd.DataFrame(aux_test, index=X_test.index, columns=X_test.columns)
aux_val = scalerMinMaxKNNImputer.transform(X_val.values)
X_val_norm = pd.DataFrame(aux_val, index=X_val.index, columns=X_val.columns)

# Without some of one hot encoding variables
noOneHotXTrain = X_train.drop(X_train.filter(regex='^idprovincia').columns, axis=1)
noOneHotXVal = X_val.drop(X_val.filter(regex='^idprovincia').columns, axis=1)
noOneHotXTest = X_test.drop(X_test.filter(regex='^idprovincia').columns, axis=1)
noOneHotXTrain = noOneHotXTrain.drop(noOneHotXTrain.filter(regex='^causa').columns, axis=1)
noOneHotXVal = noOneHotXVal.drop(noOneHotXVal.filter(regex='^causa').columns, axis=1)
noOneHotXTest = noOneHotXTest.drop(noOneHotXTest.filter(regex='^causa').columns, axis=1)

# MinMaxScaler
scalerMinMaxKNNImputer = MinMaxScaler()
scalerMinMaxKNNImputer.fit(noOneHotXTrain)
aux_train = scalerMinMaxKNNImputer.transform(noOneHotXTrain.values)
X_train_minMax_noHot = pd.DataFrame(aux_train, index=noOneHotXTrain.index, columns=noOneHotXTrain.columns)
aux_test = scalerMinMaxKNNImputer.transform(noOneHotXTest.values)
X_test_minMax_noHot = pd.DataFrame(aux_test, index=noOneHotXTest.index, columns=noOneHotXTest.columns)
aux_val = scalerMinMaxKNNImputer.transform(noOneHotXVal.values)
X_val_minMax_noHot = pd.DataFrame(aux_val, index=noOneHotXVal.index, columns=noOneHotXVal.columns)


# print(X_train.head(10))
# print(X_train_std.head(10))
# print(X_train_minMax.head(10))

trainDataframes = [X_train, X_train_std, X_train_minMax, X_train_norm, noOneHotXTrain, X_train_minMax_noHot]
testDataframes = [X_test, X_test_std, X_test_minMax, X_test_norm, noOneHotXTest, X_test_minMax_noHot]
valDataframes = [X_val, X_val_std, X_val_minMax, X_val_norm, noOneHotXVal, X_val_minMax_noHot]

## Linear Regression

In [46]:
results_df = pd.DataFrame(index=['Linear Regression'], columns=['R squared', 'time'])

from sklearn.linear_model import LinearRegression
lr = LinearRegression(normalize=False, n_jobs=-1)

# [X_train, X_train_std, X_train_minMax, X_train_norm, noOneHotXTrain, X_train_minMax_noHot]
dataLr = 0
start = time()
lr.fit(trainDataframes[dataLr], y_train)
training_time = time() - start

# y_val_predicted = lr.predict(X_test)
# r2_score_lr = lr.score(X_val_norm, y_val)
#
# print('Linear regression' ,r2_score_lr)

scores = cross_val_score(lr, valDataframes[dataLr], y_val, cv=5)
acc=np.mean(scores)
results_df.loc['Linear Regression',:] = [acc, timedelta(seconds=training_time)]
print(results_df)

# Original =                0.140283
# Std =                     -5.25768e+22
# MinMax =                  -4.04218e+19
# Norm =                    -4.04218e+19
# noOneHotXTrain =          0.12905
# X_train_minMax_noHot =    0.12905

                  R squared            time
Linear Regression  0.140283  0:00:00.950382


## KNN

In [61]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsRegressor

# [X_train, X_train_std, X_train_minMax, X_train_norm, noOneHotXTrain, X_train_minMax_noHot]
dataKNN = 5

X_train_quick = trainDataframes[dataKNN].head(5000)
y_train_quick = y_train.head(5000)

res = []

if everything:

    knnR = KNeighborsRegressor()
    # distance euclidean 9
    clf = GridSearchCV(estimator=knnR,
                             param_grid = {'weights':('uniform', 'distance'),
                                                       'n_neighbors':list(range(1,30)),
                                                       'metric': ['euclidean', 'manhattan', 'chebyshev', 'minkowski'
                                                , 'wminkowski', 'seuclidean', 'mahalanobis']},
                             n_jobs=-1,
                             return_train_score=True)
    # clf = RandomizedSearchCV(estimator=knnR,
    #                          param_distributions= dict(weights = ('uniform', 'distance'),
    #                                                    n_neighbors = list(range(1, 40)),
    #                                                    metric = ['euclidean', 'manhattan', 'chebyshev', 'minkowski'
    #                                             , 'wminkowski', 'seuclidean', 'mahalanobis']),
    #                          n_jobs=-1,
    #                          return_train_score=True,
    #                          n_iter=10)
    modelCV = clf.fit(X_train_quick, y_train_quick)
    modelCV.best_params_ , modelCV.best_score_
    y_pred = modelCV.predict(testDataframes[dataKNN].head(100))
    print('Resultat test', modelCV.score(testDataframes[dataKNN], y_test))
    print('Resultat val:', modelCV.score(valDataframes[dataKNN], y_val))

    # Resultat test 0.14673539627333576
    # Resultat val: 0.2159892244417193
    # ({'metric': 'euclidean', 'n_neighbors': 9, 'weights': 'distance'}, 0.2750767350186984)

knn = KNeighborsRegressor(metric = 'euclidean', n_neighbors = 9, weights = 'distance',n_jobs=-1)
init_time = time()
knn.fit(trainDataframes[dataKNN], y_train)
training_time = time()-init_time

scores = cross_val_score(knn, valDataframes[dataKNN], y_val, cv=5)
acc=np.mean(scores)
results_df.loc['KNN',:] = [acc, timedelta(seconds=training_time)]
print(results_df)

# Original =                -0.364093
# Std =                     0.201094
# MinMax =                  -0.0994702
# Norm =                    -0.0994702
# noOneHotXTrain =          -0.364093
# X_train_minMax_noHot =    0.229668

                  R squared            time
Linear Regression  0.140283  0:00:00.950382
KNN                0.229668  0:00:00.450912


## Random Forest Regressor

In [63]:
from sklearn.ensemble import RandomForestRegressor

# [X_train, X_train_std, X_train_minMax, X_train_norm, noOneHotXTrain, X_train_minMax_noHot]
dataRfR = 0

regr = RandomForestRegressor(max_depth=2, random_state=0)
# clf = RandomizedSearchCV(estimator=regr,
#                          param_distributions= dict(criterion = ('mse', 'mae'),
#                                                    max_depth = list(range(2, 40)),
#                                                    n_estimators = list(range(30, 50))),
#                          n_jobs=-1,
#                          n_iter=1)
#
# modelCV = clf.fit(trainDataframes[dataKNN], y_train)
modelCV = regr.fit(trainDataframes[dataRfR])

from sklearn import metrics
# print('Resultat ', modelCV.score(valDataframes[dataRfR], y_val))

modelCV.best_params_ , modelCV.best_score_

scores = cross_val_score(modelCV, valDataframes[dataRfR], y_val, cv=5)
acc=np.mean(scores)
results_df.loc['RandomForestRegressor',:] = [acc, timedelta(seconds=training_time)]
print(results_df)


KeyboardInterrupt: 

## MLP

In [ ]:
from sklearn.neural_network import MLPRegressor
regr = MLPRegressor(random_state=1, max_iter=500).fit(X_train, y_train)
print(regr.predict(X_test[:2]))
print(regr.score(X_test, y_test))

In [ ]:
print(regr.score(X_val, y_val))
print(regr.score(X_train, y_train))

